In [1]:
%matplotlib inline
import os
import re
import io
import itertools
import pprint

from bokeh.io import output_notebook, show
from bokeh.layouts import gridplot
from bokeh.plotting import figure
from bokeh.models import Range1d, Legend
from bokeh.palettes import all_palettes

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import xarray as xr

import climatools.lblnew.bestfit_params as bestfits
from climatools.lblnew import setup_bestfit, setup_overlap
import climatools.lblnew.pipeline as pipe_lblnew
import climatools.cliradlw.setup as setup_cliradlw
import climatools.cliradlw.pipeline as pipe_cliradlw

import climatools.html.html as climahtml
from climatools.lblnew.dataio import *
from climatools.plot.plot import *


import IPython.display as display

In [2]:
%run param.py

In [3]:
def load_output_file(path_csv):
    '''
    Load lblnew output .csv file to xarray.Dataset
    
    Parameters
    ----------
    path_csv: str
              Path to the .csv file to be loaded.
    ds: xarray.Dataset
        Data in the input file in the form of an xarray.Dataset.
    '''
    toindex = ['i', 'band', 'pressure', 'igg', 'g']    
    df = pd.read_csv(path_csv, sep=r'\s+')
    df = df.set_index([i for i in toindex if i in df.columns])
    df = df.rename(columns={'sfu': 'flug',
                            'sfd': 'fldg',
                            'fnet': 'fnetg',
                            'coolr': 'coolrg'})
    ds = xr.Dataset.from_dataframe(df)

    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    return ds

In [4]:
def lblnew_setup(param=None):
    if 'ng_refs' in param:
        return {'setup': setup_bestfit,
                'fname_flux_crd': 'output_flux.dat',
                'fname_cool_crd': 'output_coolr.dat',
                'fname_flux_wgt': 'output_wfluxg.dat',
                'fname_cool_wgt': 'output_wcoolrg.dat'}
    else:
        return {'setup': setup_overlap,
                'fname_flux_crd': 'output_flux.dat',
                'fname_cool_crd': 'output_coolr.dat',
                'fname_flux_wgt': 'output_wflux.dat',
                'fname_cool_wgt': 'output_wcoolr.dat'}

    
def load_lblnew_data(param):
    
    fname_dsname = [('fname_flux_crd', 'ds_flux_crd'),
                    ('fname_cool_crd', 'ds_cool_crd'),
                    ('fname_flux_wgt', 'ds_flux_wgt'),
                    ('fname_cool_wgt', 'ds_cool_wgt')]
    
    d = lblnew_setup(param)
    dir_fortran = pipe_lblnew.get_dir_case(param, setup=d['setup'])
    
    data_dict = {}
    for fname, dsname in fname_dsname:
        fpath = os.path.join(dir_fortran, d[fname])
        data_dict[dsname] = load_output_file(fpath)
    return data_dict
    

    
    

In [5]:
d = load_lblnew_data(PARAM_LBLNEW)

DS_FLUX_CRD = d['ds_flux_crd']
DS_COOL_CRD = d['ds_cool_crd']
DS_FLUX_WGT = d['ds_flux_wgt']
DS_COOL_WGT = d['ds_cool_wgt']

In [6]:
DIR_FORTRAN = pipe_cliradlw.get_fortran_dir(PARAM, 
                                            setup=setup_cliradlw)

PATH_FLUX = os.path.join(DIR_FORTRAN, 'output_flux.dat')
PATH_COOL = os.path.join(DIR_FORTRAN, 'output_coolr.dat')

DS_FLUX = load_output_file(PATH_FLUX)
DS_COOL = load_output_file(PATH_COOL)

In [7]:
output_notebook()

Loading BokehJS ...

In [8]:
def fmt_cool(ds_in):
    ds = ds_in.copy(deep=True)
    if 'igg' in ds.dims:
        ds = ds.sel(igg=1)

    if 'g' in ds.dims:
        ds = ds.sum('g')
        
    if 'i' in ds.dims:                                                                                               
        ds = ds.sel(i=ds.dims['i']) 
            
    if 'band' in ds.dims:
        try:
            ds = ds.squeeze('band')
        except ValueError:
            ds = ds.sum('band')
                
    return ds['coolrg']


def pltdata_cooling_1(ds_cool=None,
                      ds_cool_crd=None, ds_cool_wgt=None):
    


    colors = all_palettes['Set1'][4]
    data = [{'label': 'CLIRAD `coolr_bands`',
             'srs': fmt_cool(ds_cool),
             'line_dash': 'dashed', 'line_width': 5,
             'color': colors[0], 'alpha': .6},
            {'label': 'CRD',
             'srs': fmt_cool(ds_cool_crd),
             'line_dash': 'solid', 'line_width': 1.5,
             'marker': 'circle', 'marker_size': 5,
             'color': colors[2], 'alpha': 1,},
            {'label': 'WGT igg=10 wgt_flux=2',
             'srs': fmt_cool(ds_cool_wgt),
             'line_dash': 'solid', 'line_width': 3,
             'marker': 'square', 'marker_size': 5,
             'color': colors[3], 'alpha': .6}]
    return data


def nice_xlims(pltdata=None, prange=None):
    
    def get_slice(srs):
        return srs.sel(pressure=slice(*prange))
    
    srss = [d['srs'] for d in pltdata]
    vmin = min([get_slice(srs).min() for srs in srss])
    vmax = max([get_slice(srs).max() for srs in srss])
    dv = (vmax - vmin) * .01
    return float(vmin - dv), float(vmax + dv)
    

def plt_cooling_bokeh(pltdata=None):
    '''
    Plot a list of cooling rate profiles using bokeh.
    '''
    ymin = 1e-2 
    ymax = 1020 
        
    p1 = figure(title="Linear pressure scale", 
                plot_width=400)        
    xmin, xmax = nice_xlims(pltdata, prange=(50, 1050))
    for d in pltdata:
        if 'marker' in d:
            getattr(p1, d['marker'])(d['srs'].values, 
                    d['srs'].coords['pressure'].values,
                    color=d['color'], alpha=.7)
        p1.line(d['srs'].values, 
                d['srs'].coords['pressure'].values,
                color=d['color'], alpha=d['alpha'], 
                line_width=d['line_width'], line_dash=d['line_dash'])
    
    p1.y_range = Range1d(ymax, ymin)
    p1.yaxis.axis_label = 'pressure [mb]'   
    p1.x_range = Range1d(xmin, xmax)
    p1.xaxis.axis_label = 'cooling rate [K/day]'
            
    p2 = figure(title='Log pressure scale', y_axis_type='log',
                plot_width=560)
    xmin, xmax = nice_xlims(pltdata, prange=(.01, 200))
    
    rs = []
    for d in pltdata:
        rd = []
        if 'marker' in d:
            r_mark = getattr(p2, d['marker'])(d['srs'].values, 
                        d['srs'].coords['pressure'].values,
                        color=d['color'], alpha=.7)
            rd.append(r_mark)
        r_line = p2.line(d['srs'].values, 
                    d['srs'].coords['pressure'].values,
                    color=d['color'], alpha=d['alpha'], 
                    line_width=d['line_width'], line_dash=d['line_dash'])
        rd.append(r_line)
      
        rs.append(rd)
        
    p2.y_range = Range1d(ymax, ymin)  
    p2.yaxis.axis_label = 'pressure [mb]'
    
    p2.x_range = Range1d(xmin, xmax)
    p2.xaxis.axis_label = 'cooling rate [K/day]'
    
    items = [(d['label'], r) for r, d in zip(rs, pltdata)]
    legend = Legend(items=items, location=(10, -30))
    legend.label_text_font_size = '8pt'
    
    p2.add_layout(legend, 'right')  
    
    show(gridplot(p1, p2, ncols=2, plot_height=500))
    

def script_plt_cooling():
    pltdata = pltdata_cooling_1(
        ds_cool=DS_COOL,
        ds_cool_crd=DS_COOL_CRD,
        ds_cool_wgt=DS_COOL_WGT)
    plt_cooling_bokeh(pltdata=pltdata)
    print('''FIGURE. Cooling rate profiles.''')

In [9]:
def fmt_flux(ds_in, ilevels=None):
    ds = ds_in.copy(deep=True)
    
    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    if ilevels:
        ds = ds.isel(pressure=ilevels)

    if 'i' in ds.dims:                                                                                               
        ds = ds.sel(i=ds.dims['i'])
        
    if 'igg' in ds.dims:
        ds = ds.sel(igg=[1])
        ds = ds.squeeze('igg').drop('igg')
    
    if 'g' in ds.dims:
        ds = ds.sum('g')
            
    if 'band' in ds.dims:
        try:
            ds = ds.squeeze('band').drop('band')
        except ValueError:
            ds = ds.sum('band')
                
    df = ds.to_dataframe()
    df = df.set_index('level', append=True)               
    return df


def tbdata_flux(ds_flux=None, ds_flux_crd=None, ds_flux_wgt=None):
    tropopauses = {'mls': 40, 'saw': 45, 'trp': 37}
    
    if PARAM['atmpro'] == PARAM['atmpro']:
        atmpro = PARAM['atmpro']
    else:
        atmpro = 'mls'
        
    ilevels = [0, tropopauses[atmpro], -1]
    
    benchmark = {'label': 'CRD',
                 'df': fmt_flux(ds_flux_crd, ilevels=ilevels)}
    
    tbs = [{'label': 'WGT igg=10 wgt_flux=1',
            'df': fmt_flux(ds_flux_wgt, ilevels=ilevels)},
           {'label': 'CLIRAD',
            'df': fmt_flux(ds_flux, ilevels=ilevels)}]
    
    data = {'benchmark': benchmark, 'others': tbs}
    return data


def tb_flux(tbdata=None):
    benchmark = tbdata['benchmark']
    others = tbdata['others']
    
    def show_tb(d=None):
        display.display(d['df'])
        print('Table.', 'Fluxes.', d['label'])
        
    def show_tbdiff(d0=None, d=None):
        df = d['df']
        df.index = d0['df'].index
        display.display(df - d0['df'])
        print('Table.', 'Fluxes.', 
              '({}) - ({})'.format(d['label'], d0['label']))
        
    show_tb(benchmark)
    for d in others:
        show_tb(d)
        
    for d in others:
        show_tbdiff(d=d, d0=benchmark)
        
        
def script_tb_flux():
    data = tbdata_flux(ds_flux=DS_FLUX,
                       ds_flux_crd=DS_FLUX_CRD,
                       ds_flux_wgt=DS_FLUX_WGT)
    tb_flux(tbdata=data)

In [10]:
def band_map():
    '''
    Maps spectral bands in lblnew to spectral bands in clirad.
    '''
    lblnew2clirad = {'1': '1',
            '2': '2',
            '3a': '3',
            '3b': '4',
            '3c': '5',
            '4': '6', 
            '5': '7',
            '6': '8',
            '7': '9', 
            '8': '10',
            '9': '11'}
    
    clirad2lblnew = {clirad: lblnew 
                     for lblnew, clirad in lblnew2clirad.items()}
    return clirad2lblnew


def gasbands():
    bmap = band_map()
    gases = list(PARAM['molecule'].keys())
    
    bands = [bmap[str(b)] for b in PARAM['band']]
    return list(itertools.product(gases, bands))


def print_bestfit_params():
    print('Best-fit values for each (gas, band)')
    print('------------------------------------')
    for gas, band in gasbands():
        print('o {} band{}'.format(gas, band))
        param = bestfits.kdist_params(molecule=gas, band=band)
        for name, value in sorted(param.items()):
            print(2 * ' ' + '{} = {}'.format(name, value))

        
def print_input_param():        
    for name, value in PARAM.items():
        print('{} = {}'.format(name, value))
        
        
def print_lblnew_param():
    for name, value in sorted(PARAM_LBLNEW.items()):
        print('{} = {}'.format(name, value))


In [11]:
'''
Cooling rate profile by layer
'''
def script_coolr_bylayer():
    '''
    Displays cooling rate of each layer
    '''
    keys = ['CRD', 'CLIRAD `coolr_bands`']
    dss = [DS_COOL_CRD, DS_COOL]
    das = [fmt_cool(ds) for ds in dss]
    
    dfs = [da.to_dataframe() for da in das]
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'coolr_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))



In [12]:
'''
Fluxes by layer
'''
def script_flux_bylayer():
    '''
    Display fluxes of each layer
    '''
    keys = ['CRD', 'CLIRAD']
    dss = [DS_FLUX_CRD, DS_FLUX]
    dfs = [fmt_flux(ds) for ds in dss]
    
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'fluxes_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))


In [13]:
def show_html(s):
    display.display(display.HTML(s))

    
def show_markdown(s):
    display.display(display.Markdown(s))

    
def script():
    
    title = '### CLIRAD-LW'
    s_input = 'Input Parameters'
    s_lblnew_param = 'LBLNEW case compaired against'
    s_bestfits = 'Best-fit Parameters'
    s_plt_cooling = 'Cooling Rate Profiles'
    s_tb_flux = 'Flux Comparison'
    s_coolr_bylayer = 'Cooling Rate by Layer'
    s_flux_bylayer = 'Fluxes by Layer'
    
    # title
    show_markdown('### CLIRAD-LW')
    
    # TOC
    show_markdown('### Table of Contents')
    show_html(climahtml.getHTML_hrefanchor(s_input))
    show_html(climahtml.getHTML_hrefanchor(s_lblnew_param))
    show_html(climahtml.getHTML_hrefanchor(s_plt_cooling))
    show_html(climahtml.getHTML_hrefanchor(s_tb_flux))
    show_html(climahtml.getHTML_hrefanchor(s_bestfits))
    show_html(climahtml.getHTML_hrefanchor(s_coolr_bylayer))
    show_html(climahtml.getHTML_hrefanchor(s_flux_bylayer))
    
    # Input parameters
    show_html(climahtml.getHTML_idanchor(s_input))
    show_markdown(climahtml.getMarkdown_sectitle(s_input))
    print_input_param()
    
    # LBLNEW parameters 
    show_html(climahtml.getHTML_idanchor(s_lblnew_param))
    show_markdown(climahtml.getMarkdown_sectitle(s_lblnew_param))
    print_lblnew_param()
    
    # Cooling rate profiles
    show_html(climahtml.getHTML_idanchor(s_plt_cooling))
    show_markdown(climahtml.getMarkdown_sectitle(s_plt_cooling))
    script_plt_cooling()
    
    # Flux comparison
    show_html(climahtml.getHTML_idanchor(s_tb_flux))
    show_markdown(climahtml.getMarkdown_sectitle(s_tb_flux))
    script_tb_flux()
    
    # Best-fit values
    show_html(climahtml.getHTML_idanchor(s_bestfits))
    show_markdown(climahtml.getMarkdown_sectitle(s_bestfits))
    print_bestfit_params()
    
    # Cooling rate by layer
    show_html(climahtml.getHTML_idanchor(s_coolr_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_coolr_bylayer))
    script_coolr_bylayer()
    
    # Fluxes by layer
    show_html(climahtml.getHTML_idanchor(s_flux_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_flux_bylayer))
    script_flux_bylayer()
   
    

In [14]:
script()

### CLIRAD-LW

### Table of Contents

# Input Parameters

commitnumber = 523cbb7
molecule = {'ch4': 1.8e-06}
band = [9]
atmpro = trp
tsfc = 300


# LBLNEW case compaired against

...

Best-fit values for each (gas, band)
------------------------------------
o ch4 band7
  atmpro = mls
  band = 7
  commitnumber = a06b618
  conc = 1.8e-06
  dv = 0.001
  klin = 2e-21
  molecule = ch4
  ng_adju = [0, 0]
  ng_refs = [2, 3]
  nv = 1000
  option_compute_btable = 0
  option_compute_ktable = 1
  option_wgt_flux = 1
  option_wgt_k = 1
  ref_pts = [(1, 250), (500, 250)]
  tsfc = 294
  vmax = 1380
  vmin = 1215
  w_diffuse = [(1.66, 1.66), (1.66, 1.66, 1.66)]
  wgt = [(0.7, 0.7), (0.4, 0.6, 0.75)]


# Cooling Rate by Layer

CRD           CLIRAD `coolr_bands`          
            layer    coolrg                layer    coolrg
pressure                                                  
0.000312        1 -0.088991                    1 -0.050306
0.000750        2 -0.088160                    2 -0.049779
0.001052        3 -0.086363                    3 -0.048569
0.001476        4 -0.084528                    4 -0.047382
0.002070        5 -0.082682                    5 -0.046139
0.002904        6 -0.080857                    6 -0.044913
0.004074        7 -0.079076                    7 -0.043623
0.005714        8 -0.077342                    8 -0.042326
0.008015        9 -0.075708                    9 -0.041033
0.011243       10 -0.074210                   10 -0.039703
0.015771       11 -0.072926                   11 -0.038399
0.022122       12 -0.071943                   12 -0.037060
0.031031       13 -0.071387                   13 -0.035782
0.043528       14 -0.070818                   14 -0.034079
0.061057       15 -0.063031                   15 -0.027213
0.085645       16 -0.042391                   16 -0.011334
0.120136       17 -0.013049                   17  0.011325
0.168516       18  0.022051                   18  0.039131
0.236378       19  0.063626                   19  0.072859
0.331549       20  0.112730                   20  0.113581
0.465100       21  0.170968                   21  0.162386
0.652400       22  0.232279                   22  0.213931
0.915100       23  0.244432                   23  0.230592
1.283650       24  0.204489                   24  0.206925
1.800600       25  0.154026                   25  0.169573
2.525700       26  0.102591                   26  0.124783
3.542800       27  0.064214                   27  0.084437
4.969550       28  0.039655                   28  0.052380
6.970850       29  0.024887                   29  0.029482
9.778100       30  0.016230                   30  0.015336
13.715850      31  0.010830                   31  0.008400
19.239350      32  0.006799                   32  0.005469
26.987250      33  0.003117                   33  0.003434
37.855300      34 -0.000717                   34  0.000155
53.100050      35 -0.007758                   35 -0.007702
73.887500      36 -0.017468                   36 -0.019192
97.662500      37 -0.025833                   37 -0.029051
121.437500     38 -0.029457                   38 -0.033307
145.212500     39 -0.029192                   39 -0.033265
168.987500     40 -0.029401                   40 -0.033157
192.762500     41 -0.029395                   41 -0.032710
216.537500     42 -0.029066                   42 -0.031996
240.312500     43 -0.028406                   43 -0.030938
264.087500     44 -0.027432                   44 -0.029664
287.862500     45 -0.026163                   45 -0.028170
311.637500     46 -0.024619                   46 -0.026413
335.412500     47 -0.022816                   47 -0.024571
359.187500     48 -0.020771                   48 -0.022473
382.962500     49 -0.018493                   49 -0.020200
406.737500     50 -0.015989                   50 -0.017749
430.512500     51 -0.013266                   51 -0.015129
454.287500     52 -0.010324                   52 -0.012339
478.062500     53 -0.007165                   53 -0.009244
501.837500     54 -0.003780                   54 -0.006007
525.612500     55 -0.000157                   55 -0.002424
549.387500     56  0.003722                   56  0.001356
573.162500     57  0.007883                   57  0.005516
596.937500     58  0.012367                   58  0.009957
620.712500     59  0.017207                   59  0.014731
644.487500     60  0.022404                   60  0.019816
668.262500     61  0.027770                   61  0.025009
692.037500     62  0.032711                   62  0.029762
715.812500     63  0.036315                   63  0.033272
739.587500     64  0.038203                   64  0.035387
763.362500     65  0.039100                   65  0.036622
78

# Fluxes by Layer

CRD                          CLIRAD                \
                        flug      fldg      fnetg       flug          fldg   
pressure    level                                                            
0.000000    1     -21.608790  0.000000 -21.608790 -21.640402  9.028008e-08   
0.000624    2     -21.608799  0.000003 -21.608796 -21.640408  1.868106e-06   
0.000876    3     -21.608803  0.000004 -21.608799 -21.640410  2.593790e-06   
0.001229    4     -21.608808  0.000005 -21.608803 -21.640413  3.637475e-06   
0.001723    5     -21.608815  0.000008 -21.608807 -21.640417  5.136751e-06   
0.002417    6     -21.608825  0.000011 -21.608814 -21.640423  7.291459e-06   
0.003391    7     -21.608839  0.000016 -21.608824 -21.640432  1.038621e-05   
0.004757    8     -21.608859  0.000022 -21.608836 -21.640443  1.483457e-05   
0.006672    9     -21.608886  0.000032 -21.608854 -21.640459  2.122600e-05   
0.009359    10    -21.608924  0.000046 -21.608878 -21.640481  3.040474e-05   
0.013128    11    -21.608977  0.000065 -21.608911 -21.640512  4.358944e-05   
0.018415    12    -21.609050  0.000094 -21.608957 -21.640555  6.251254e-05   
0.025830    13    -21.609154  0.000134 -21.609020 -21.640615  8.967944e-05   
0.036232    14    -21.609298  0.000190 -21.609108 -21.640698  1.286348e-04   
0.050823    15    -21.609501  0.000271 -21.609230 -21.640813  1.848690e-04   
0.071291    16    -21.609778  0.000395 -21.609383 -21.640967  2.723949e-04   
0.100000    17    -21.610134  0.000606 -21.609527 -21.641156  4.228236e-04   
0.140271    18    -21.610563  0.000974 -21.609590 -21.641367  6.886586e-04   
0.196760    19    -21.611049  0.001607 -21.609442 -21.641572  1.154864e-03   
0.275997    20    -21.611525  0.002680 -21.608845 -21.641698  1.965133e-03   
0.387100    21    -21.611838  0.004477 -21.607361 -21.641598  3.360066e-03   
0.543100    22    -21.611643  0.007441 -21.604202 -21.640977  5.740118e-03   
0.761700    23    -21.610320  0.012132 -21.598187 -21.639358  9.660970e-03   
1.068500    24    -21.607707  0.018403 -21.589304 -21.636593  1.527714e-02   
1.498800    25    -21.604221  0.025340 -21.578881 -21.632957  2.219021e-02   
2.102400    26    -21.600063  0.032195 -21.567868 -21.628597  2.995606e-02   
2.949000    27    -21.595667  0.038088 -21.557579 -21.623847  3.772133e-02   
4.136600    28    -21.591248  0.042703 -21.548546 -21.618876  4.462982e-02   
5.802500    29    -21.586905  0.046185 -21.540720 -21.613779  4.987042e-02   
8.139200    30    -21.582777  0.048945 -21.533832 -21.608687  5.294002e-02   
11.417000   31    -21.579098  0.051568 -21.527530 -21.603848  5.405633e-02   
16.014700   32    -21.576302  0.054671 -21.521631 -21.599717  5.450062e-02   
22.464000   33    -21.575206  0.058768 -21.516437 -21.597212  5.617390e-02   
31.510500   34    -21.577275  0.064179 -21.513097 -21.598122  6.076451e-02   
44.200100   35    -21.584969  0.070794 -21.514175 -21.605816  6.869122e-02   
62.000000   36    -21.606509  0.075977 -21.530532 -21.630409  7.704424e-02   
85.775000   37    -21.657249  0.077523 -21.579726 -21.689758  8.233676e-02   
109.550000  38    -21.731933  0.079454 -21.652479 -21.775955  8.670732e-02   
133.325000  39    -21.825922  0.090481 -21.735441 -21.881760  9.869823e-02   
157.100000  40    -21.932441  0.114787 -21.817654 -21.999863  1.231077e-01   
180.875000  41    -22.050169  0.149711 -21.900458 -22.128596  1.584507e-01   
204.650000  42    -22.177964  0.194721 -21.983243 -22.266565  2.042882e-01   
228.425000  43    -22.314774  0.249673 -22.065101 -22.412734  2.603358e-01   
252.200000  44    -22.459623  0.314522 -22.145101 -22.565913  3.263762e-01   
275.975000  45    -22.611601  0.389242 -22.222359 -22.725249  4.021594e-01   
299.750000  46    -22.769850  0.473808 -22.296042 -22.889896  4.874631e-01   
323.525000  47    -22.933562  0.568186 -22.365376 -23.058852  5.820251e-01   
347.300000  48    -23.101963  0.672329 -22.429634 -23.231643  6.856092e-01   
371.075000  49    -23.274310  0.786178 -22.4881

In [15]:
display.HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')